In [1]:
import numpy as np
import pandas as pd
import importlib
import sys
# import keras
from tqdm import tqdm
import pickle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.optimizers import SGD
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.utils import class_weight
from sklearn.utils import resample
from collections import defaultdict

from classifiers.cnn import Classifier_CNN
from classifiers.fcn import Classifier_FCN
from classifiers.resnet import Classifier_RESNET
importlib.reload(sys.modules[Classifier_CNN.__module__])
importlib.reload(sys.modules[Classifier_FCN.__module__])
importlib.reload(sys.modules[Classifier_RESNET.__module__])

<module 'classifiers.resnet' from '/home/aasleptsov98/Coursework/cnn_vs_rnn/classifiers/resnet.py'>

In [4]:
data_name = "installments_payments.data.pkl"

In [5]:
app_train = pd.read_csv("input/application_train.csv")
instalments_payments = pd.read_csv("input/installments_payments.csv")

In [6]:
instalments_payments = instalments_payments.fillna(0)

In [8]:
ohe = preprocessing.OneHotEncoder()
categories = np.array(list(set(instalments_payments["NUM_INSTALMENT_VERSION"].astype(float).values))).reshape(-1,1)
ohe.fit(categories)

OneHotEncoder()

In [9]:
encoded = np.array(ohe.transform(instalments_payments["NUM_INSTALMENT_VERSION"].values.reshape(-1,1)).todense())
temp_df = pd.DataFrame(encoded, columns=range(encoded.shape[1]))

In [10]:
instalments_payments = pd.concat([instalments_payments.drop(["NUM_INSTALMENT_VERSION"], axis=1), temp_df], axis=1)

In [11]:
n_timesteps = 40
n_features = instalments_payments.shape[1]

In [12]:
targets = dict()
for index, row in tqdm(app_train.iterrows()):
    targets[row["SK_ID_CURR"]] = row["TARGET"]

307511it [00:56, 5403.77it/s]


In [ ]:
X = []
y = []

for id_cur, df_id_cur in tqdm(instalments_payments.groupby(by=["SK_ID_PREV", "SK_ID_CURR"])):
    if len(df_id_cur) > 0 and id_cur[1] in targets:
        X.append(df_id_cur.sort_values(by=['MONTHS_BALANCE'])[-n_timesteps:].drop(["SK_ID_PREV", "SK_ID_CURR"], axis=1).values)
        y.append(targets[id_cur[1]])

In [ ]:
with open(data_name, "wb") as f:
    pickle.dump((X, y), f)

In [ ]:
lens = []
for data in X:
    lens.append(len(data))
print(max(lens), min(lens), sum(lens) / len(lens))

In [ ]:
for i in range(len(X)):
    X[i] = np.append(np.zeros((max(n_timesteps - X[i].shape[0], 0), X[i].shape[1]), dtype=int), X[i], axis=0)

In [21]:
X = np.array(X)
y = np.array(y)
bincount_classes = np.bincount(y)
weights = {0: bincount_classes[0] / np.max(bincount_classes), 1: bincount_classes[1] / np.max(bincount_classes)}

In [ ]:
#upsample
minority_class = X[y==1]
bc = np.bincount(y)
upsampled_minority_class = resample(minority_class, replace=True, n_samples=bc[0]-bc[1], random_state=123)
upsampled_X = np.append(X, upsampled_minority_class, axis=0)
upsampled_y = np.append(y, np.ones(bc[0]-bc[1], dtype=int))
weights = {0: 1, 1: 1}

In [22]:
train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))

In [29]:
batch_size = 64
epochs = 30

# FCN

In [ ]:
model_fcn = Classifier_FCN(X[0].shape, nb_classes=2)
model_fcn.fit(X_train, y_train, X_val, y_val, batch_size=batch_size, epochs=epochs)

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 80, 27)]          0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 80, 128)           27776     
_________________________________________________________________
batch_normalization_3 (Batch (None, 80, 128)           512       
_________________________________________________________________
activation_3 (Activation)    (None, 80, 128)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 80, 256)           164096    
_________________________________________________________________
batch_normalization_4 (Batch (None, 80, 256)           1024      
_________________________________________________________________
activation_4 (Activation)    (None, 80, 256)          

In [ ]:
y_pred_fcn = model_fcn.predict(X_test)
roc_auc_score(y_test, y_pred_fcn)

# ResNet

In [ ]:
model_resnet = Classifier_RESNET(X[0].shape, nb_classes=2, verbose=True)
model_resnet.fit(X_train, y_train, X_val, y_val)

In [ ]:
y_pred_resnet = model_resnet.predict(X_test)
roc_auc_score(y_test, y_pred_resnet)